In [28]:
############### Pro-IDD : Pareto-based ensemble for imbalanced and drifting data streams ####################
################################# Authors: Muhammad Usman, Huanhuan Chen ####################################
############################## For any queries: muhammadusman@mail.ustc.edu.cn ##############################
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import preprocessing
from skmultiflow.trees import HoeffdingTreeClassifier
from time import perf_counter
from sklearn import clone
from sklearn.base import BaseEstimator
from sklearn.metrics import accuracy_score
import sys
import scipy.stats as ss
from scipy.stats import ks_2samp
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.cluster import AgglomerativeClustering
import statistics
from decimal import Decimal
import scipy.cluster.hierarchy as sch
from sklearn import preprocessing
from numpy import sqrt
import matplotlib.ticker as mticker
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
import random
from sklearn.ensemble import VotingClassifier
from sklearn.preprocessing import LabelEncoder
import math 
from numpy.linalg import norm
import statistics
import random
import os
import warnings
from sklearn.metrics import roc_auc_score
from sklearn.metrics.pairwise import euclidean_distances
from scipy.spatial  import distance_matrix
from scipy.spatial.distance import pdist,squareform
from scipy.cluster.hierarchy import linkage
import pygmo as pg
import matplotlib.pyplot as plt
import numpy as np
from pygmo import *
from timeit import default_timer as timer

pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.8f}'.format)
#np.set_printoptions(suppress=True)
#np.set_printoptions(threshold=1000000000)
warnings.filterwarnings('ignore')


def __get_coefficients(y_true, y_pred_a, y_pred_b):
    a, b, c, d = 0, 0, 0, 0
    for i in range(y_true.shape[0]):
        if y_pred_a[i] == y_true[i] and y_pred_b[i] == y_true[i]:
            a = a + 1
        elif y_pred_a[i] != y_true[i] and y_pred_b[i] == y_true[i]:
            b = b + 1
        elif y_pred_a[i] == y_true[i] and y_pred_b[i] != y_true[i]:
            c = c + 1
        else:
            d = d + 1
    return a, b, c, d



def agreement_measure(y_true, y_pred_a, y_pred_b):
    a, b, c, d = __get_coefficients(y_true, y_pred_a, y_pred_b)
    agreement = 1
    try:
        agreement = (a + b + c + d) / float(b + c)
    except:
        agreement = 1
    return agreement

def listToString(s): 
    
    # initialize an empty string
    str1 = "" 
    
    # traverse in the string  
    for ele in s: 
        str1 += str(ele)+ "-" 
    
    # return string  
    return str1 
# normalize a vector to have unit norm
def normalize(weights):
    # calculate l1 vector norm
    result = norm(weights, 1)
    # check for a vector of all zeros
    if result == 0.0:
        return weights
    # return normalized vector (unit norm)
    return weights / result
def pltcolor(lst):
    cols=[]
    for l in lst:
        if l==0:
            cols.append('blue')
        elif l==1:
            cols.append('red')
    return cols



def detect_overlap(data_min,data_maj,n_features,current_window):
    global arr_win_x
    fishers = []
    found = 0
    max_f1 = 0
    max_feature_index = 0
    
   
    Xy_up = data_min
    Xy_down = data_maj
    frames = [Xy_up,Xy_down]
    Xy_d = pd.concat(frames)
    
    f_data_up_means = Xy_up.mean()
    f_data_down_means= Xy_down.mean()
    
    f_data_up_vars = Xy_up.var()
    f_data_down_vars =Xy_down.var()
    
    f2_val = 1
    for i in range(n_features):
        mean_up = f_data_up_means[i]
        mean_down = f_data_down_means[i]
        var_up = f_data_up_vars[i]
        var_down = f_data_down_vars[i]
        fisher = ((mean_up-mean_down)**2) / (var_up + var_down)
        fishers.append([i,round(fisher,5),round(mean_up,5),round(mean_down,5)])
        if(fisher>max_f1) :
            max_f1 = fisher
            max_feature_index = i

    max_index= max_feature_index + 1
    #print("Actual Max:",max_f1)
    max_f1 = 1/(1+max_f1)   # push between (0,1]
    #print("Transformed Max:",max_f1)       
    fishers=sorted(fishers,key=lambda x: x[1], reverse=False)
    return round(max_f1,5),round(f2_val,5), fishers


def min_plusplus(X_maj,X_min,y_maj,y_min,n_features,overlap_val,ups,downs):
        global overlap_threshold_f1
        global output_folder
        global imbalance_ratio_threshold
        
        
        ##################### Update Imbalance status of current window
        is_imbalance = False
        ir = 0
        if(ups ==0 or downs ==0):
            ir = 99 # very high, always considered imbalanced
        elif(ups > downs):
            ir = ups/downs
        elif(ups < downs):
            ir = downs/ups
        elif(ups == downs):
            ir = 1
        arr_ir.append(ir)
        if(ir>=imbalance_ratio_threshold):
            is_imbalance = True
            
            
        ######################################## Execure Min++ functions        
        if(is_imbalance == False): #nothing to be done if there is no imbalance            
            frames = [X_min,X_maj]
            modified_X = pd.concat(frames)
            frames = [y_min,y_maj]        
            modified_Y = pd.concat(frames)
            return modified_X.to_numpy(), modified_Y.to_numpy().reshape(-1)
                
                        
        if(is_imbalance==True): # 1) fix disjuncts and overlaps if there is imbalance
            
            #Get updated minority data
            modified_Xmin,modified_Ymin = fix_disjuncts(X_min,y_min,(len(X_maj)-len(X_min)))
            
            #see if majority is to be fixed by removing overlaps
            modified_Xmaj= X_maj
            modified_Ymaj = y_maj
            
            is_overlap = False            
            if (overlap_val > overlap_threshold_f1): 
                is_overlap =  True

            if is_overlap:
                modified_Xmaj = modified_Xmaj.reset_index(drop=True)
                modified_Xmin = modified_Xmin.reset_index(drop=True)
                idx_maj = len(modified_Xmaj)
                all_data = pd.concat([modified_Xmaj,modified_Xmin])
                a = pdist(all_data,metric='euclidean')
                t_l = len(all_data)
                d_1 = linkage(a)
                d_2 = pd.DataFrame(d_1,columns=['i1','i2','i3','i4'])
                filtered_values = np.where( ((d_2["i1"] < idx_maj) & (d_2["i2"] > idx_maj) & (d_2["i2"] <t_l)  ) | ((d_2["i1"] > idx_maj) & (d_2["i1"] < t_l ) & (d_2["i2"] < idx_maj) ) )
                filtered =  d_2.loc[filtered_values]
                arr_indexes = filtered.iloc[:,0].to_numpy() 
                to_remove = round(len(arr_indexes)*disjunct_threshold_perc_maj)
                modified_Xmaj.drop(index=arr_indexes,inplace=True)
                modified_Ymaj = modified_Ymaj.drop(modified_Ymaj.index[:to_remove])
                
            
        #combine minority and majority
        frames = [modified_Xmin,modified_Xmaj]
        modified_X = pd.concat(frames)
        frames = [modified_Ymin,modified_Ymaj]
        modified_Y = pd.concat(frames)
            
        #return modified data
        return modified_X.to_numpy(), modified_Y.to_numpy().reshape(-1)

def fix_disjuncts(X,y,samples_to_add):
    global disjuncts
    global output_folder
    global disjuncts_threshold
    global arr_ord_cols

    X = X.reset_index(drop=True)
    y = y.reset_index(drop=True)
    data_scaled = X
    n_samples, n_features = X.shape
    try:
        data_scaled = preprocessing.scale(X)
    except:
        return X,y
    linked = linkage(data_scaled, 'average')
    last = linked[-10:, 2]
    last_rev = last[::-1]
    idxs = np.arange(1, len(last) + 1)
    accele = np.diff(last, 2)  # 2nd derivative of the distances
    accele_rev = accele[::-1]
    k = accele_rev.argmax() + 2  # if idx 0 is the max of this we want 2 clusters
    #k = random.randint(2,7)
    model = AgglomerativeClustering(affinity='euclidean', compute_full_tree='auto', connectivity=None, linkage='average', memory=None, n_clusters=k)
    y_hc=model.fit_predict(X)
    aggloclust=model.fit(X)
    
    (unique, counts) = np.unique(aggloclust.labels_, return_counts=True)
    frequencies = np.asarray((unique, counts)).T
    frequencies = frequencies[np.argsort(frequencies[:, 1])[::-1]]

    arr_cumsum = np.cumsum(frequencies,axis=0)
    cumsum = arr_cumsum[:,1]
    arr_sum_cumsum = np.sum(frequencies,axis=0)
    arr_clusters = []
    sum_cumsum = arr_sum_cumsum[1]
    for i in range(len(arr_cumsum)):
        val = cumsum[i]
        arr_clusters.append(frequencies[i,0])

    
    arr_y_frames  = []
    arr_x_frames  = []
    cluster1_count =0
    df_x1 = []
    df_y1 = []
    disjuncts = len(arr_clusters)
    if(disjuncts<disjuncts_threshold):
        return X,y
    
    valid_clusters =0
    for i in range(len(arr_clusters)):
        arr_indexes = ((np.where(y_hc == int(arr_clusters[i]))))
        df_y = y.loc[y.index[arr_indexes]]
        if(len(df_y) > 2):
            valid_clusters += 1
    if(valid_clusters == 0):
        return X,y
    each_cluster_addition = int(np.round(samples_to_add / valid_clusters ))
    
    for i in range(len(arr_clusters)):
            arr_indexes = ((np.where(y_hc == int(arr_clusters[i]))))
            df_x = X.loc[X.index[arr_indexes]]
            df_y = y.loc[y.index[arr_indexes]]
            
            
            if(len(df_x) <3): #could be outliers + avoid exception in sample generation
                continue
            
            to_add = 0
            if(i==0):
                cluster1_count = len(df_x)
                to_add = each_cluster_addition                
            else:
                to_add = each_cluster_addition
                
                #make equal size clusters in minority
                current_cluster_count = len(df_x)                    
                to_add += (cluster1_count-current_cluster_count)
            
            df_cov = df_x.cov()
            df_mean = df_x.mean(axis=0)
            new_samples = np.random.multivariate_normal(df_mean, df_cov,to_add)
            
            cols = []
            for f in range(1,n_features+1):
                cols.append(str(f))
            df_new_samples = pd.DataFrame(new_samples,columns=cols)
            df_x = pd.concat([df_x.reset_index(drop=True),df_new_samples.reset_index(drop=True)],axis=0)
            df_x = df_x.reset_index(drop=True)
            
            #round values for ordinal cols
            if(len(arr_ord_cols)>0):
                for h in range (0,len(arr_ord_cols)):
                    ix = arr_ord_cols[h]
                    df_x[str(ix)] = round(df_x[str(ix)]) 
            for j in range(0,to_add):
                df_y = df_y.append(pd.Series([1],index=['class']),ignore_index = True) 
            df_y = df_y.reset_index(drop=True)
            
            arr_x_frames.append(df_x)
            arr_y_frames.append(df_y)
        
    
    np.set_printoptions(threshold=np.inf)            
    modified_X = pd.concat(arr_x_frames)
    modified_Y = pd.concat(arr_y_frames)
    
    modified_X = modified_X.reset_index(drop=True)
    modified_Y = modified_Y.reset_index(drop=True)

    return modified_X,modified_Y    


class DetectorClassifier(BaseEstimator):
    def __init__(self, clf, classes):
        if not hasattr(clf, "partial_fit"):
            raise TypeError("Choose incremental classifier")
        self.clf = clf
        
        self.classes = classes
        self.change_detected = 0

    def fit(self, X, y):
        #self.clf = clone(self.clf)
        self.clf.fit(X, y)
        return self

    def partial_fit(self,modified_X,modified_Y,fullFit):
            if(fullFit):
               # print("^^^^Full Fit^^^")    
                self.clf = self.clf.fit(modified_X, modified_Y, classes=self.classes)            
            else:
               # print("^^^^Partial Fit^^^")
                self.clf = self.clf.partial_fit(modified_X, modified_Y, classes=self.classes)

            return "", self
    
    ##### Prdict Function        
    def predict(self, X):
        return np.asarray(self.clf.predict(X), dtype=np.int64).ravel()
    
    ##### Score Function
    def score(self, X,y):
        return self.clf.score(X,y)
    
    def predict_proba(self, X):
        return self.clf.predict_proba(X) 
    
    
    
def prequential(window_size,Xy,X, y, n_features,file_out_prefix):
    
    """Prequential Evaluation: First test the incoming instances and prepare a batch to train the model again.
    """
        
    arr_clf = []
    arr_clf_temp_to_use = []
    arr_weights = []
    arr_recall0_cw = []
    arr_recall1_cw = []
    
    a= 0.00001
    

    global arr_recall_0
    global arr_recall_1
    global output_folder    
    global arr_recall
    global arr_specificity
    global maj_weight
    global min_weight
    global overlap_threshold
    global imbalance_ratio_threshold
    global disjunct_threshold_perc_maj
    global pareto_indices
    global ensemble_pool_size
    
    pareto_indices = []
    arr_actual_win = []
    arr_pred_win = []
    arr_roc = []
    arr_divs = []
    ensemble_tp = 0
    ensemble_fp =0
    ensemble_fn = 0
    ensemble_tn = 0
    disjuncts = 0
    
    ensemble_correct =0
    zeroClassCounter = 0
    oneClassCounter = 0
    current_correct_0 = 0
    current_correct_1 = 0
    
    current_correct_ensemble_0 = 0
    current_correct_ensemble_1 = 0
    
    predictions = ""
    
    
    row_num = y.shape[0]
    
    # Split an init batch for training of same size as of batch
    X_init = X[0:w]
    y_init = y[0:w]
    
    # Used for prequentail testing and training
    X_train = X[0:]
    y_train = y[0:]    
    Xy_train = Xy[0:]
    
    #Due to K=3 in Equation 6. If more histor is required, add more variables here. 
    #Non Dynamic Code
    recall_0_history_1 = np.zeros(ensemble_pool_size)
    recall_0_history_2 = np.zeros(ensemble_pool_size)
    recall_0_history_3 = np.zeros(ensemble_pool_size)
    recall_1_history_1 = np.zeros(ensemble_pool_size)
    recall_1_history_2 = np.zeros(ensemble_pool_size)
    recall_1_history_3 = np.zeros(ensemble_pool_size)
    
    
    ensemble_preds = np.zeros(row_num-w)
    current_correct_0_cw = np.zeros(ensemble_pool_size)
    current_correct_1_cw = np.zeros(ensemble_pool_size)
    recall_0_cw = np.zeros(ensemble_pool_size)
    recall_1_cw = np.zeros(ensemble_pool_size)
    
    
    arr_temp = []

    
    #Create classifiers and add to the pool trained on initial batch
    for g in range (0,1):
        obj_clf = DetectorClassifier(HoeffdingTreeClassifier(), np.unique(y))
        obj_clf = obj_clf.fit(X_init, y_init)
        arr_clf.append(obj_clf)
        arr_weights.append(1) 
        pareto_indices=np.append(pareto_indices,0) 
        
    current_w = 0
    window_counter = 0
    global prev_win_x 
    global prev_win_y 
    strProcessing = ""
    correct_predictions = 0

    strout = ""
    strgmean = ""

    oneClassCounter_CWindow = 0
    zeroClassCounter_CWindow = 0
    arr_cw_ens_preds = []
    current_correct_0_cw_est = 0
    current_correct_1_cw_est = 0
    window_recall_0 = 0
    window_recall_1 = 0
    arr_accuracy = []
    
    current_ensemble = []
    current_weights = []
    current_ensemble_use = []
    
    weights_cal_flag = True  #Send true first time to prepare default weights
    
    for i in range(0, row_num - n_train):
        arr_x.append(i)
        current_w += 1
        start_time = perf_counter()
        
        arr_preds = []
        arr_temp_preds = []
        
        #predict using each base classifier to calculate recalls
        for n in range(0,len(arr_clf)):
            clf = arr_clf[n]
            prediction_clf = clf.predict(X_train[i, :].reshape(1, -1))
            arr_preds.append(prediction_clf[0])
            arr_temp_preds.append(prediction_clf[0])
        
        arr_cw_ens_preds.append(arr_temp_preds)            
       
        #weights are re-calculated only if batch is ready
        #the flag is set upon start of training, and once the batch is ready
        if( weights_cal_flag == True):            
            weights_cal_flag = False ## no repeat unless new batch is ready
            
            df_weights_arr = pd.DataFrame(arr_weights)            
            df_weights_arr = df_weights_arr.sort_values(ascending=False,by=df_weights_arr.columns[0])
            arr_clf_temp_to_use = []
            estimators_arr_temp = []
            arr_weights_temp = []
                
            if(len(arr_clf) < ensemble_pool_size):
                for b in range (0,len(arr_clf)):
                    arr_clf_temp_to_use.append(arr_clf[b])
                    estimators_arr_temp.append(("HT-" + str(b), arr_clf[b]))
                    arr_weights_temp.append(1)     
            else:                    
                ## get weights of preto-selected classifiers
                for h in range(0,len(pareto_indices)) :
                    try:
                        indexx = pareto_indices[h].astype(int)
                    except:
                        indexx = pareto_indices[h]   
                    index_original = df_weights_arr.iloc[[h]].index.values[0]            
                    arr_clf_temp_to_use.append(arr_clf[indexx])
                    estimators_arr_temp.append(("HT-" + str(index_original), arr_clf[indexx]))
                    arr_weights_temp.append(arr_weights[indexx])        
            
            
        
        #Ensemble Prediction
        pred = 0
        eclf = VotingClassifier(estimators = estimators_arr_temp, voting='hard', weights=arr_weights_temp) 
        eclf.estimators_ = arr_clf_temp_to_use
        eclf.le_ = LabelEncoder().fit(np.unique([0, 1]))
        pred = eclf.predict(X_train[i, :].reshape(1, -1))
        arr_pred_win.append(pred)
        
        
        lambda_0 = maj_weight #Majority
        lambda_1 = min_weight #Minority
        
        if y_train[i] == 0:
            zeroClassCounter += 1
            zeroClassCounter_CWindow += 1
        else: 
            oneClassCounter += 1
            oneClassCounter_CWindow += 1
        
        
        for n in range(0,len(arr_clf)):
            if(y_train[i] == 0 and arr_preds[n] ==0):
                current_correct_0_cw[n] = current_correct_0_cw[n]+1
            if(y_train[i] == 1 and arr_preds[n] ==1):    
                current_correct_1_cw[n] = current_correct_1_cw[n]+1
        
        correct_found = 0  
        y_actual = 0
        y_pred = 0
        if(y_train[i] ==0 and pred ==0):
            correct_found +=1
            current_correct_ensemble_0 += 1
            current_correct_0_cw_est += 1
            ensemble_tn += 1
            y_actual = 0
            y_pred = 0
        if(y_train[i] ==1 and pred == 1):
            correct_found +=1
            current_correct_ensemble_1 += 1
            current_correct_1_cw_est += 1
            ensemble_tp += 1
            y_actual = 1
            y_pred = 1
        if(y_train[i] ==1 and pred ==0):
            ensemble_fn += 1
            y_actual = 1
            y_pred = 0
        if(y_train[i] ==0 and pred == 1):
            ensemble_fp += 1
            y_actual = 0
            y_pred = 1
        if(correct_found ==1):
            ensemble_correct += 1
            
        arr_actual_win.append(y_actual)
        
        #when the batch is ready or an in complete batch at the end
        if( ( current_w % window_size == 0)  or (current_w != window_size and i == (row_num - w -1)) ):
            arr_win_x.append(window_counter)
            window_counter += 1
            
            ##window-based evaluation measures calculation
            window_recall_0 = 0
            window_recall_1 = 0
            if zeroClassCounter_CWindow != 0:
                window_recall_0 =  current_correct_0_cw_est / zeroClassCounter_CWindow
            if oneClassCounter_CWindow != 0:
                window_recall_1 =  current_correct_1_cw_est / oneClassCounter_CWindow
            
            window_gmean = sqrt(window_recall_0*window_recall_1)
            arr_gmean.append(window_gmean*100)
            arr_recall_0.append(window_recall_0*100)
            arr_recall_1.append(window_recall_1*100)            
            
            specificity = 0
            recall = 0
            try:
                specificity = ensemble_tn/(ensemble_tn+ensemble_fp)
            except:
                specificity = 0                        
            try:
                recall = ensemble_tp/(ensemble_tp+ensemble_fn)
            except:
                recall = 0
            
            arr_specificity.append(specificity*100)
            arr_recall.append(recall*100)
                      
            ## Historical Recalls 
            if zeroClassCounter_CWindow != 0:
                for n in range(0,len(arr_clf)):
                    recall_0_cw[n] =  current_correct_0_cw[n] / zeroClassCounter_CWindow                      
                    if(window_counter>2): #Due to K=3 in Equation 6
                        recall_0_history_1[n] = recall_0_history_2[n]
                        recall_0_history_2[n] = recall_0_history_3[n]
                        recall_0_history_3[n] = recall_0_cw[n]
                    current_correct_0_cw[n] =0    
            if oneClassCounter_CWindow != 0:
                for n in range(0,len(arr_clf)):
                    recall_1_cw[n] =  current_correct_1_cw[n] / oneClassCounter_CWindow
                    if(window_counter>2):#Due to K=3 in Equation 6
                        recall_1_history_1[n] = recall_1_history_2[n]
                        recall_1_history_2[n] = recall_1_history_3[n]
                        recall_1_history_3[n] = recall_1_cw[n]                        
                    current_correct_1_cw[n] =0
                    
            arr_weights_temp_out=[]
            arr_diversity_temp_out = []
            arr_recall_0_temp_out = []
            arr_recall_1_temp_out  = []
            arr_final_weight_temp_out = []            
            
            for n in range(0,len(arr_clf)):
                weight = 0
                if(window_counter < 3): #for first 2 windows only
                    weight = (lambda_0 * recall_0_cw[n]) + (lambda_1*recall_1_cw[n])
                else: #window 3 onwards
                    #0.2,0.3,0.5 are used in Equation 6 for K = 3. For different values of K, change this code
                    recall_0_hist_value =  0.2*recall_0_history_1[n]+  0.3*recall_0_history_2[n]+  0.5*recall_0_history_3[n]
                    recall_1_hist_value =  0.2*recall_1_history_1[n]+  0.3*recall_1_history_2[n]+  0.5*recall_1_history_3[n]
                    weight = (lambda_0 * recall_0_hist_value) + (lambda_1*recall_1_hist_value)
                
                arr_weights[n] = weight                
                arr_weights_temp_out.append(weight)                
                arr_recall_0_temp_out.append(recall_0_cw[n])
                arr_recall_1_temp_out.append(recall_1_cw[n])
                arr_final_weight_temp_out.append(arr_weights[n])
            
            
            '''### diversity work '''
            arr_temp = np.vstack(arr_cw_ens_preds)            
            arr_divs = []            
            for m in range(0,len(arr_clf)):
                a_sum = 0
                for n in range(0,len(arr_clf)):
                    if(m == n):
                        continue
                    ens_preds_a = arr_temp[:,m]
                    ens_preds_b = arr_temp[:,n]
                    agr = 1
                    try:
                        agr = agreement_measure(y_preq, ens_preds_a, ens_preds_b)
                    except: 
                        agr = 1
                    a_sum += agr
                if(len(arr_clf) != 1):
                    #average over all 
                    avg = (2*a_sum)/((len(arr_clf))*(len(arr_clf)-1))                    
                else:
                    avg=1
                arr_divs.append(avg)                
            '''### diversity work ends here '''
            
            arr_cw_ens_preds = []            
            oneClassCounter_CWindow = 0
            zeroClassCounter_CWindow = 0  
            current_correct_0_cw_est = 0
            current_correct_1_cw_est = 0
            ensemble_tn = 0
            ensemble_tp = 0
            ensemble_fp = 0
            ensemble_fn = 0
            
            for n in range(0,len(arr_clf)):
                current_correct_0_cw[n]=0
                current_correct_1_cw[n] = 0
            
            
            #print("**************************************************************************************")
            print("\nCurrent Window:", window_counter )
            #print("From:",str(i-(window_size-1)))
            #print("To:",str(i))
            #print("Current Ensemble Size:",len(arr_clf))
            #print("*******")
            
            x_preq = X_train[i-(window_size-1):i]
            y_preq = y_train[i-(window_size-1):i]

            Xy_preq = Xy_train[i-(window_size-1):i]
            ups =  np.count_nonzero(y_preq == 1)
            downs = np.count_nonzero(y_preq == 0)
            
            arr_clf_temp = []

            ####################### <Overlap detection> ############################################
            cols_str = []
            for f in range(1,n_features+1):
                cols_str.append(str(f))
            cols_str.append('class')
            Xy_d = pd.DataFrame(Xy_preq, columns = cols_str)


            if(ups < downs):
                Xy_min = Xy_d[(Xy_d['class'] == 1)]
                Xy_maj = Xy_d[(Xy_d['class'] == 0)]
            if(ups>downs):
                Xy_min = Xy_d[(Xy_d['class'] == 0 )]
                Xy_maj = Xy_d[(Xy_d['class'] == 1 )]

            cols = []
            for f in range(0,n_features):
                cols.append(f)
            cols_class = [n_features]
            X_min = Xy_min[Xy_min.columns[cols]]
            X_maj = Xy_maj[Xy_maj.columns[cols]]
            y_min = Xy_min[Xy_min.columns[cols_class]]
            y_maj = Xy_maj[Xy_maj.columns[cols_class]]
            
            frames = [X_min,X_maj]
            modified_X = pd.concat(frames)
            frames = [y_min,y_maj]
            modified_Y = pd.concat(frames)

            cols_num = []            
            for p in range(0,n_features):
                cols_num.append(p)
            m_X = modified_X
            m_Y = modified_Y            
            m_X.columns = cols_num
            m_Y.columns = [0]
            overlap_val,f2_val, fishers = detect_overlap(X_min, X_maj,n_features,window_counter)   
            ####################### </ Overlap Detection> ############################################
            
            ####################### Balance this batch using Min++ ###################################
            mod_x,mod_y = min_plusplus(X_maj,X_min,y_maj,y_min,n_features,overlap_val,ups,downs)                    
            
            
            
            ## Partial Fit the previous classifiers                        
            arr_temp_clfs = []   
            for n in range(0,len(arr_clf)):
                clf = arr_clf[n]                    
                status,clf_temp = clf.partial_fit(mod_x,mod_y,False)                
                arr_temp_clfs.append(clf_temp)
            arr_clf = arr_temp_clfs
            
            
            ######################## Pareto Calculation ##################################
            points_for_preto = []
            for n in range(0,len(arr_clf)):
                points_for_preto.append([-1*arr_weights[n],arr_divs[n]])            
            pareto_indices = []            
            pts = pg.non_dominated_front_2d(points =points_for_preto)
            
            for r in range (0,len(pts)):
                pareto_indices.append(pts[r])
            #print("diversities:",arr_divs)
            #print("weights:",arr_weights)
            #print("pareto indices:",pareto_indices)    
            
            #remove one classifier if the pool has reached.. 
            if(len(arr_clf) > ensemble_pool_size-1):
                preto_min_index = 0
                
                least_weight = arr_weights[0]
                for q in range (1,len(arr_weights)):
                    w_q = arr_weights[q]
                    if(w_q < least_weight):                        
                        preto_min_index = q
                        least_weight = w_q
                

                arr_clf = np.delete(arr_clf,preto_min_index)
                arr_weights = np.delete(arr_weights,preto_min_index)
                arr_divs = np.delete(arr_divs,preto_min_index)
                for g in range (preto_min_index,len(arr_clf)):
                    recall_0_history_1[g] =  recall_0_history_1[g+1]
                    recall_0_history_2[g] = recall_0_history_2[g+1]
                    recall_0_history_3[g] = recall_0_history_3[g+1]
                    recall_1_history_1[g] = recall_1_history_1[g+1]
                    recall_1_history_2[g] = recall_1_history_2[g+1]
                    recall_1_history_3[g] = recall_1_history_3[g+1]
                    current_correct_0_cw[g] = current_correct_0_cw[g+1] 
                    current_correct_1_cw[g] =  current_correct_1_cw[g+1]
                        
            
            ######################### add new classifier to the pool #####################
            obj_clf1 = DetectorClassifier(HoeffdingTreeClassifier(), np.unique(y))
            status,clf_temp = obj_clf1.partial_fit(mod_x,mod_y,True) # True here means FULL FIT
            arr_weights = np.append(arr_weights,0)
            arr_divs = np.append(arr_divs,1)
            arr_clf = np.append(arr_clf,clf_temp)
            
            current_w = 0
            prev_win_x = x_preq
            prev_win_y =  y_preq
            weights_cal_flag = True
            strgmean = ""
            print("---\n Avg. Gmean: ", np.mean(arr_gmean))
            print("Avg. Specificity: ", np.mean(arr_specificity))
            print("Avg. Recall: ", np.mean(arr_recall))

    #uncomment this line to output the file with specificity, recall and gmeans
    #np.savetxt(output_folder + file_out_prefix + "-" + str(window_counter) +'.csv',  np.c_[arr_recall,arr_specificity,arr_gmean], delimiter=',',fmt='%1.2f')

    return np.mean(arr_gmean),np.mean(arr_recall),np.mean(arr_specificity)

def read_data(filename):
    df = pd.read_csv(filename)
    data = df.values
    return data, data[:, :-1], data[:, -1]

def delete_files_from_folder(folder):
    for filename in os.listdir(folder):
        file_path = os.path.join(folder, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))

            
#*********************************  Main ******************************** #
global arr_overlap 
global imbalance_threshold
global arr_recall_0
global arr_recall_1
global arr_gmean
global f1_overlap_value
global maj_weight
global min_weight
global arr_ord_cols
global arr_specificity
global arr_recall
global output_folder
global stream_folder
global pareto_indices
global ensemble_pool_size

f1_overlap_value = 0

cwd = os.path.dirname(os.getcwd())
output_folder = cwd +  '/output/'
stream_folder = cwd +  '/streams/'
#delete_files_from_folder(output_folder)

arr_recall = []
arr_specificity = []
arr_gmean = []
arr_recall_0 = []
arr_recall_1 = []
arr_recall_0_nb = []
arr_recall_1_nb = []
arr_x = []


ensemble_pool_size = 15  #pool size to be used
w =1500  #window/batch size
overlap_threshold_f1 =0.5 #overlap threshold for detection
disjuncts_threshold =2 #min minority disjuncts to consider
disjunct_threshold_perc_maj = 0.5 #overlap removal threshold for majority
maj_weight = 0.5 #weight for majority class recall in ensemble
min_weight = 0.5 #weight for minority class recall in ensemble

arr_ord_cols= []
arr_ord_cols = [6,7,8,9,10] #1-based index  GMSC
#arr_ord_cols = [13,14,15,16,17,18,19,20,21] #1-based index  IJCNN1
#arr_ord_cols = [10,11,12] #1-based index  LOAN
#arr_ord_cols = [9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28] #1-based index  PKADD


#real world data sets
#filename = "ijcnn1-full-30oct2022.csv"
#filename = "cod-rna-311022.csv"
#filename = "MiniBooNE_PID_Mod.csv"
filename =  "GMSC.csv"
#filename = "loan_191122.csv"
#filename = "PAKDD.csv"

#synthetic data sets 
#filename = "v5_imb.csv"
#filename = "v9_imb.csv"
#filename = "moa-sea-50K12-g.csv"
#filename = "moa-sea-50K12-a.csv"
#filename = "moa-sea-50K12-gr.csv"
#filename = "moa-sea-50K12-ar.csv"
#filename = "moa-sine-50K12-g.csv"
#filename = "moa-sine-50K12-a.csv"
#filename = "moa_sine_5k_12.csv"
#filename = "moa_sine_abrupt_5k_12.csv"

filename = stream_folder  + filename

Xy, X, y = read_data(filename)
# Set x,y as numeric
X = X.astype(float)
n_samples, n_features = X.shape
print("samples:",n_samples)
print("features:",n_features)
ups =  np.count_nonzero(y == 1)
downs = np.count_nonzero(y == 0)
print("1s:",str(ups/n_samples))
print("0s:",str(downs/n_samples))

arr_gmean_iteration = []
arr_recall_iteration = []
arr_specificity_iteration = []
arr_times = []
for k in range (0,10):
    arr_specificity = []
    arr_recall = [] 
    arr_recall_0= []
    arr_recall_1= []
    arr_gmean= []
    print("------------------------------------------------*******************------------------------------------------")
    print("iteration no:",k)
    for i in range(len(clfs)):
        file_out_prefix = "ijcnn1-pareto-" + str(ensemble_pool_size) + "-" + str(maj_weight) + "-" + str(min_weight) + "-" + str(disjuncts_threshold) + "-" + str(overlap_threshold_f1) + "-" + str(disjunct_threshold_perc_maj)

        start = timer()
        file_out_prefix1 = file_out_prefix + "-" + str(k)
        gmean,recall,specificity = prequential(w,Xy,X, y, n_features,file_out_prefix1)
        arr_gmean_iteration.append(gmean)
        arr_recall_iteration.append(recall)            
        arr_specificity_iteration.append(specificity)
        end = timer()
        print("time:",end-start)
        arr_times.append(end-start)
    
   

print("---Iterations Averages---")
print("Avg. Gmean: ",np.mean(arr_gmean_iteration))
print("Avg. Recall: ",np.mean(arr_recall_iteration))
print("Avg. Specificity: ",np.mean(arr_specificity_iteration))
print("Avg. Time: ",np.mean(arr_times))
sys.exit()       
